In [1]:
import scipy as sp
import scipy.interpolate
import numpy as np
import glob
from tabulate import tabulate #to export in table format
import scipy.interpolate as interp

In [2]:
sn_names=glob.glob("*.dat")
print(len(sn_names))

113


In [3]:
file = open("factors.txt")
lines = file.readlines()
file.close()

In [4]:
max_names = []
max_values = []

In [5]:
for line in lines:
    if line.startswith('#'): continue
    co=line.rstrip().replace('INDEF','Nan').split()

    max_names.append(co[0])
    max_values.append(co[1])

In [6]:
new_names = []

In [7]:
x = np.linspace(-10,50,61)
y = np.linspace(3000,9000,601)

In [8]:
X = []

In [9]:
max_hist = []

In [10]:
for k in range(0,len(sn_names)):
    
    if sn_names[k][:-4] in max_names:
        
        idx = max_names.index(sn_names[k][:-4])
        #print(float(max_values[idx]))
    
        file = open(sn_names[k])
        lines = file.readlines()
        file.close()

        xtemp = []

        for line in lines:

            if line.startswith('#'): continue
            co=line.rstrip().replace('INDEF','Nan').split()

            xtemp.append(co[0])

        xtemp = np.array(xtemp, dtype=float)
        
        xtemp = xtemp*float(max_values[idx])
        

        #xtemp = xtemp/max(xtemp)
        
        

        if max(xtemp) < 100000:
            
            new_names.append(sn_names[k])
            z = np.reshape(xtemp, (601, 61))

            zz = []
            xx = []
            yy = []

            for i in range(0,len(y)):

                for j in range(0,len(x)):

                    zz.append(z[i][j])
                    xx.append(x[j])
                    yy.append(y[i])

            X.append(np.array(zz))


In [11]:
len(X)

95

In [12]:
import os.path

In [13]:
x_grid = np.linspace(-10,50,61)
y_grid = np.linspace(3000,9000,601)
B1, B2 = np.meshgrid(x_grid, y_grid, indexing='xy')

In [14]:
med = []
sec_mom = []
th_mom = []
four_mom = []

In [15]:
from scipy.stats import skew

In [16]:
save_path = '/home/joao/Documentos'

In [17]:
for i in range(0,len(X[0])):
    
    templist_0 = []
    templist = []
    #templist2 = []
    count = 0
    count_2 = 0
    for j in range(0,len(X)):
        
        if X[j][i] > 0:
            
            templist_0.append(X[j][i])
            count += 1
    '''
    if count > 5:
        
        for h in range(0, len(templist_0)):

            if np.mean(templist_0) - np.sqrt(np.var(templist_0)) < templist_0[h] < np.mean(templist_0) + np.sqrt(np.var(templist_0)):

                templist.append(templist_0[h])

                count_2 += 1
    '''
    if count > 5:

        media = np.mean(templist_0)
        mom_2 = 0
        mom_3 = 0
        mom_4 = 0
        for k in range(0, len(templist_0)):

            mom_2 += (templist_0[k] - media)**2
            mom_3 += (templist_0[k] - media)**3
            mom_4 += (templist_0[k] - media)**4


        #variance = np.var(templist)
        med.append(media)
        sec_mom.append(mom_2/count)
        th_mom.append(mom_3/count)
        four_mom.append(mom_4/count)

    else:

        med.append(0)
        sec_mom.append(0)
        th_mom.append(0)
        four_mom.append(0)


In [18]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel

In [19]:
kernel = 0.5 * (RBF(length_scale=5, length_scale_bounds=(1, 10)) + WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-5,0.5)))

In [20]:
z = np.reshape(med, (601, 61))

exp_x = []
exp_y = []
exp_z = []

for i in range(0,len(y)):

    for j in range(0,len(x)):

        exp_z.append(z[i][j])
        exp_x.append(x[j])
        exp_y.append(y[i])


In [21]:
list_data = []
for j in range(0, len(y)):
    
    temp_x = []
    temp_z = []
    
    for i in range(0, len(exp_y)):

        if exp_y[i] == y[j]:

            temp_x.append([exp_x[i]])
            temp_z.append(exp_z[i])
            
    gp = GaussianProcessRegressor(kernel = kernel, alpha=0.0).fit(temp_x, temp_z)        
    y_mean = gp.predict(x[:, np.newaxis])     
    
    list_data.append(y_mean)

In [22]:
temp_x = []
temp_y = []
temp_z = []

for i in range(0, len(x)):
    
    for j in range(0, len(y)):
        
        temp_x.append(x[i])
        temp_y.append(y[j])
        temp_z.append(list_data[j][i])

In [23]:
table = []
for l in range(0,len(temp_x)):
    table.append((temp_x[l],temp_y[l],temp_z[l]))


completeName = os.path.join(save_path, "med.dat")

f = open(completeName, 'w')
f.write(tabulate(table, tablefmt="plain"))
f.close()

In [24]:
z = np.reshape(sec_mom, (601, 61))

exp_x = []
exp_y = []
exp_z = []

for i in range(0,len(y)):

    for j in range(0,len(x)):

        exp_z.append(z[i][j])
        exp_x.append(x[j])
        exp_y.append(y[i])


In [25]:
list_data = []
for j in range(0, len(y)):
    
    temp_x = []
    temp_z = []
    
    for i in range(0, len(exp_y)):

        if exp_y[i] == y[j]:

            temp_x.append([exp_x[i]])
            temp_z.append(exp_z[i])
            
    gp = GaussianProcessRegressor(kernel = kernel, alpha=0.0).fit(temp_x, temp_z)        
    y_mean = gp.predict(x[:, np.newaxis])     
    
    list_data.append(y_mean)

In [26]:
temp_x = []
temp_y = []
temp_z = []

for i in range(0, len(x)):
    
    for j in range(0, len(y)):
        
        temp_x.append(x[i])
        temp_y.append(y[j])
        temp_z.append(list_data[j][i])

In [27]:
table = []
for l in range(0,len(temp_x)):
    table.append((temp_x[l],temp_y[l],temp_z[l]))


completeName = os.path.join(save_path, "sec_mom.dat")

f = open(completeName, 'w')
f.write(tabulate(table, tablefmt="plain"))
f.close()

In [28]:
z = np.reshape(th_mom, (601, 61))

exp_x = []
exp_y = []
exp_z = []

for i in range(0,len(y)):

    for j in range(0,len(x)):

        exp_z.append(z[i][j])
        exp_x.append(x[j])
        exp_y.append(y[i])


In [29]:
list_data = []
for j in range(0, len(y)):
    
    temp_x = []
    temp_z = []
    
    for i in range(0, len(exp_y)):

        if exp_y[i] == y[j]:

            temp_x.append([exp_x[i]])
            temp_z.append(exp_z[i])
            
    gp = GaussianProcessRegressor(kernel = kernel, alpha=0.0).fit(temp_x, temp_z)        
    y_mean = gp.predict(x[:, np.newaxis])     
    
    list_data.append(y_mean)

In [30]:
temp_x = []
temp_y = []
temp_z = []

for i in range(0, len(x)):
    
    for j in range(0, len(y)):
        
        temp_x.append(x[i])
        temp_y.append(y[j])
        temp_z.append(list_data[j][i])

In [31]:
table = []
for l in range(0,len(temp_x)):
    table.append((temp_x[l],temp_y[l],temp_z[l]))


completeName = os.path.join(save_path, "th_mom.dat")

f = open(completeName, 'w')
f.write(tabulate(table, tablefmt="plain"))
f.close()